In [ ]:
# including the project directory to the notebook level
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import numpy as np
import warnings  

In [ ]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)
    from keras import backend as K
    from nn_source import models as mp
from dataprocess import dataprocessor as dp

In [ ]:
# read the pickled file for ahu data
ahudata = dp.readfile('../data/processed/ahu1energy.pkl')

# return pickled df
ahu = ahudata.return_df(processmethods=['file2df'])

# read the pickled file for ghi data
ghidata = dp.readfile('../data/processed/ghi.pkl')

# return pickled df
ghi = ghidata.return_df(processmethods=['file2df'])

In [ ]:
# selecting only required columns and rearranging them
ahu = ahu[[
    'AHU_1 outdoorAirTemp', 'WeatherDataProfile humidity',
    'AHU_1 supplyAirTemp', 'HW_BTU_METER currentKbtuDeltaReading',
    'CHW_BTU_METER currentKbtuDeltaReading'
]]

# renaming columns
ahu.columns = ['oat', 'orh', 'sat', 'hwe', 'cwe']

# Total energy is sum of heating and coling
ahu['totale'] = ahu['hwe'] + ahu['cwe']
# dropping heating and cooling energy columns
ahu = ahu.drop(columns=['hwe', 'cwe'])

In [ ]:
# merging ahu and ghi data
df = dp.merge_df_columns([ahu, ghi])

# rearranging columns
df = df[['oat', 'orh', 'sat', 'Ghi', 'totale']]

In [ ]:
df.head()

In [ ]:
df['HourSine'] = np.sin(2*np.pi*df.index.hour/24.0)
df['HourCos'] = np.cos(2*np.pi*df.index.hour/24.0)
df['WeekSine'] = np.sin(2*np.pi*df.index.week/52.0)
df['WeekCos'] = np.cos(2*np.pi*df.index.week/52.0)
df['DayofWeekSine'] = np.sin(2*np.pi*df.index.dayofweek/7.0)
df['DayofWeekCos'] = np.cos(2*np.pi*df.index.dayofweek/7.0)

In [ ]:
df = df[[
    'oat', 'orh', 'sat', 'Ghi', 'HourSine', 'HourCos', 'WeekSine', 'WeekCos',
    'DayofWeekSine', 'DayofWeekCos', 'totale'
]]

In [ ]:
df.head()

In [ ]:
# Creating a list of 7 day dataframes for training
dflist = dp.df2dflist(df, subsequence=True, period=1, days=7, hours=0)

In [ ]:
len(dflist)

In [ ]:
X_train, X_test, y_train, y_test, X_scaler, y_scaler = dp.df2arrays(
    dflist[1],
    predictorcols=['oat', 'orh', 'sat', 'Ghi'],
    outputcols=['totale'],
    scaling=True,
    feature_range=(0,1),
    reshaping=True,
    lag=-1)

In [ ]:
for i in [X_train, X_test, y_train, y_test]:
    print(i.shape)

In [ ]:
!rm -rf ../results/*

In [ ]:
# create the results directory
try:
    os.mkdir('../results/lstmtrain/')
except FileExistsError:
    files = os.listdir('../results/lstmtrain/')
    for f in files:
        os.remove('../results/lstmtrain/' + f)
        
os.mkdir('../results/lstmtrain/loginfo')

In [ ]:
try:
    del model
except NameError:
    pass

K.clear_session()

In [ ]:
#Instantiate learner model
model = mp.lstm_model('../results/lstmtrain/',
                      inputdim=X_train.shape[-1],
                      outputdim=1,
                      period=1)

# Desing model architecture
model.design_model(lstmhiddenlayers=[16] * 1,
                   densehiddenlayers=[32] * 5,
                   dropoutlist=[[], []],
                   batchnormalizelist=[[], []])

In [ ]:
model.show_model()

In [ ]:
# creating early sstopping, model checkpointing and learning reate changing callbacks
model.model_callbacks()

In [ ]:
model.train_model(X_train, y_train, X_test, y_test, epochs = 100)

In [ ]:
#!tensorboard --logdir '../results/lstmtrain/loginfo/'

In [ ]:
preds_train, preds_test = model.evaluate_model(X_train,
                                               y_train,
                                               X_test,
                                               y_test,
                                               y_scaler,
                                               scaling = True,
                                               saveplot=True)

In [ ]:
dp.dataframeplot(dflist[2], lazy=False, style='-', legend=True)